In [16]:
#coding=utf-8
import csv
import re  #分词
import numpy as np
import pandas as pd

def get_key (dict, value):
    return [k for k, v in dict.items() if v == value]

csv_reader = csv.reader(open('data/match_station.csv', encoding='utf-8'))
Dict = {}
for row in csv_reader:  #0 城区监测点　１　郊区监测点
    Dict[row[0]] = row[1]

In [ ]:
#找对应项，构建列表
csv_reader1 = csv.reader(open('data/Beijing_historical_meo_grid.csv', encoding='utf-8'))
#临近气象站点列表
another_factor = {}
for s0 in Dict:
    for s1 in csv_reader1:
        data_list = []
        if Dict[s0] == s1[0]:
            key = s0 + "#" + s1[3]
            for i in range(4, 7):
                data_list.append(s1[i])
            another_factor[key] = data_list

In [30]:
test_reader = csv.reader(open('data/beijing_17_18_aq.csv', encoding='utf-8'))
#给测试集站点填加温度　湿度信息
temper = []
press  = []
humi   = []
for s0 in test_reader:
    if s0[0] == 'stationId':
        continue
    key = s0[0] + "#" + s0[1]
    if key in another_factor.keys():
        temper.append(another_factor[key][0])
        press.append(another_factor[key][1])
        humi.append(another_factor[key][2])
    else:  #出现缺失的情况，补一个缺失值
        temper.append(np.nan)
        press.append(np.nan)
        humi.append(np.nan)

In [34]:
df = pd.read_csv('data/beijing_17_18_aq.csv')
df['temperature'] = temper
df['pressure'] = press
df['humidity'] = humi
df.to_csv("233.csv")